In [1]:
import papermill as pm
import numpy as np
import pandas as pd
from os import mkdir, listdir
from os.path import exists, isfile, join
import json

In [2]:
def load_config(config_path = 'config.json'):
    file = open(config_path,'r')
    return json.load(file)

In [3]:
config = load_config('config.json')

In [4]:
config

{'img_height': 128,
 'img_width': 192,
 'epochs': 2,
 'batch_size': 64,
 'remove_background': False,
 'need_training': True,
 'prepare_datasets': True,
 'datasets_paths': [['/usr/data/shared_directory/team_1/data/Tomato_leaf_disease',
   'data_transforms/tomato_leaf_diseases_transform.ipynb']],
 'do_augmentation': True,
 'model_name': 'tomato_test',
 'model_json_path': None,
 'model_weights_path': None}

In [5]:
output_notebooks_dir = 'output_notebooks'

if not exists(output_notebooks_dir):
    mkdir(output_notebooks_dir) 
    
def craft_output_notebook_name(notebook_name):
    name = notebook_name.split('/')[-1]
    return join(output_notebooks_dir,name)

In [6]:
# запускаем трансформ датасетов

if config['prepare_datasets']:
    
    datasets_paths = config['datasets_paths']
    
    for (dataset_path,notebook_path) in datasets_paths:
        pm.execute_notebook(
            input_path = notebook_path,
            output_path = craft_output_notebook_name(notebook_path),
            parameters=dict(dataset_path = dataset_path,
                           output_path = 'data_transforms'),
            nest_asyncio=True
        )

In [7]:
# запускаем file rearrange

if config['prepare_datasets']:
    
    if not exists('data'):
        mkdir('data')
        
    pm.execute_notebook(
        input_path = 'file_rearrange.ipynb',
        output_path = craft_output_notebook_name('file_rearrange.ipynb'),
        parameters = dict(csv_folder = 'data_transforms',
                         output_path_pathologies = 'data',
                         output_path_plants = 'data'),
        nest_asyncio=True
    )
    
#train_pathology.csv
#test_pathology.csv
#train_plants.csv
#test_plants.csv

In [8]:
# запускаем copy_resize
if config['prepare_datasets']:
    
    pm.execute_notebook(
        input_path = 'copy_resize.ipynb',
        output_path = craft_output_notebook_name('copy_resize.ipynb'),
        parameters = dict(root = 'data'),
        nest_asyncio=True
    )

KeyboardInterrupt: 

In [ ]:
# запусксем тренировку модели

pm.execute_notebook(
    input_path = 'train_model.ipynb',
    output_path = craft_output_notebook_name('train_model.ipynb'),
    parameters = dict(
        train_pathology_path = 'data/train_pathology.csv',
        test_pathology_path = 'data/test_pathology.csv',
        train_plants_path = 'data/train_plants.csv',
        test_plants_path = 'data/test_plants.csv'
    ),
    nest_asyncio=True
)

In [ ]:
#запускаем репорт

model_name = config['model_name']

pm.execute_notebook(
    input_path = 'report.ipynb',
    output_path = craft_output_notebook_name('report.ipynb'),
    parameters = dict(
        model_name = config['model_name'],
        model_path = join('output',model_name,'model.json'),
        weights_path = join('output',model_name,'model_weights.h5'),
        history_path = join('output',model_name,'history'),
        csv_path = 'data/test_pathology.csv'
    ),
    nest_asyncio=True
)

In [ ]:
#сохраняем конфиг
import shutil
shutil.copy('config.json',join('output',model_name,'config.json'))

#сохраняем описание эксперимента
shutil.copy('description.txt',join('output',model_name,'description.txt'))

In [ ]:
#запускаем блокнот с визуализацией

viz_data = 'data/test_pathology.csv'

pm.execute_notebook(
    input_path = 'visualization.ipynb',
    output_path = join('output',model_name,'visualization.ipynb'),
    parameters = dict(
        model_name = config['model_name'],
        model_path = join('output',model_name,'model.json'),
        weights_path = join('output',model_name,'model_weights.h5'),
        csv_path = viz_data,
        augment_on_predict = False
    ),
    nest_asyncio=True
)

In [ ]:
#запускаем блокнот с подтверждением результатов эксперимента

pm.execute_notebook(
    input_path = 'confirm_experiment.ipynb',
    output_path = craft_output_notebook_name('confirm_experiment.ipynb'),
    parameters = dict(table_path = 'experiments/experiments_table.csv',
                     model_name = model_name
                     ),
    nest_asyncio=True
)
